# Generador de votos

Este simulador de votos lee la tabla de votos y emite un voto cada intervalo de tiempo. Además actualiza el número de votos pendientes que quedan por emitir para cada municipio y cada partido. Idealmente, los votos aparecerían en el stream sin intervalos fijos y podrían llegar en rachas, pero gracias a Kafka podemos escribir la aplicación de procesado del stream sin tener en cuenta esas consideraciones.

In [ ]:
import pandas as pd
from confluent_kafka import Producer
import time
import json
import random
import uuid

from ejercicios.votes import SEED, TOPIC_VOTES, DATA
random.seed(SEED)

In [ ]:
ds = pd.read_csv(DATA)
ds.head()

In [ ]:
def choose_partition(num_partitions, key):
    if num_partitions == 1 or key is None:
        return 0
    else:
        try:
            as_num = int(key[0], base=16)
            return as_num % num_partitions
        except:
            return 0


In [ ]:
ds['Votos'].sum()

In [ ]:
p = Producer({'bootstrap.servers': 'localhost:9092'})
m = None
def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        pass

while True:
    # Trigger any available delivery report callbacks from previous produce() calls
    p.poll(0)
    topic_list = p.list_topics(TOPIC_VOTES)
    topic_metadata = topic_list.topics[TOPIC_VOTES]
    num_partitions = len(topic_metadata.partitions.keys())
    
    vote_id = str(uuid.uuid4())

    if ds.size == 0:
        print('All votes sent!')
        break

    vote = ds.sample(n=1)
    m = vote
    as_dict = vote.to_dict('records')[0]
    num_votos = as_dict['Votos']
    as_dict.pop('Votos')

    p.produce(TOPIC_VOTES,
              key=vote_id,
              partition=choose_partition(num_partitions, vote_id),
              value=json.dumps(as_dict).encode('utf-8'),
              callback=delivery_report)
    
    if num_votos == 1:
        ds = ds.drop(index=vote.index, axis=0)
        print('Removing {}'.format(as_dict))
    else:
        ds.at[vote.index, 'Votos'] = num_votos - 1

    time.sleep(1)

p.flush()



In [ ]:
ds['Votos'].sum()